# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
#import libraries
import sys
import numpy as np
import pandas as pd
import nltk
from sqlalchemy import create_engine
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report,f1_score
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df =pd.read_sql_table('DisasterResponse',engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
     # replace url from the data with 'urlplaceholder' 
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    #converting sentence to individual words
    words=word_tokenize(text.lower())
    # lemmatize the words
    lemi=WordNetLemmatizer()
    words=[lemi.lemmatize(word) for word in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline1 = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])
pipeline2 = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=5,shuffle=True)
pipeline1.fit(x_train,y_train)
pipeline2.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred1=pipeline1.predict(x_test)
y_pred2=pipeline2.predict(x_test)

In [21]:
print(classification_report(y_test.values,y_pred1,labels=[0,1],target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      3690
               request       0.85      0.69      0.76      1150
                 offer       0.00      0.00      0.00        26
           aid_related       0.83      0.91      0.87      2730
          medical_help       0.74      0.31      0.44       499
      medical_products       0.87      0.20      0.33       335
     search_and_rescue       0.63      0.14      0.22       162
              security       0.20      0.01      0.02       119
              military       0.83      0.32      0.46       223
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.50      0.64       387
                  food       0.91      0.58      0.71       759
               shelter       0.88      0.45      0.60       589
              clothing       0.79      0.22      0.34       100
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [22]:
print(classification_report(y_test.values,y_pred2,labels=[0,1],target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      3690
               request       0.85      0.71      0.77      1150
                 offer       0.00      0.00      0.00        26
           aid_related       0.83      0.92      0.87      2730
          medical_help       0.77      0.44      0.56       499
      medical_products       0.74      0.37      0.49       335
     search_and_rescue       0.55      0.21      0.30       162
              security       0.31      0.08      0.12       119
              military       0.78      0.48      0.60       223
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.76      0.79       387
                  food       0.93      0.81      0.87       759
               shelter       0.82      0.63      0.71       589
              clothing       0.79      0.55      0.65       100
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
parameters = {
    'tfidf__sublinear_tf':[False,True],
    'clf__estimator__n_estimators':[10,50,100],
    'clf__estimator__learning_rate':[0.1,0.5,1],
}

cv = GridSearchCV(pipeline2,param_grid=parameters,n_jobs=4,cv=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
cv.fit(x_train,y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'tfidf__sublinear_tf': [False, True], 'clf__estimator__n_estimators': [10, 50, 100], 'clf__estimator__learning_rate': [0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
ypred=cv.predict(x_test)

In [31]:
print(classification_report(y_test.values,ypred,labels=[0,1],target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      3690
               request       0.86      0.73      0.79      1150
                 offer       0.00      0.00      0.00        26
           aid_related       0.83      0.94      0.88      2730
          medical_help       0.79      0.36      0.49       499
      medical_products       0.79      0.32      0.46       335
     search_and_rescue       0.62      0.17      0.27       162
              security       0.27      0.03      0.06       119
              military       0.82      0.43      0.56       223
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.80      0.81       387
                  food       0.94      0.81      0.87       759
               shelter       0.85      0.59      0.70       589
              clothing       0.85      0.55      0.67       100
                 money       0.86      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [32]:
cv.best_params_

{'clf__estimator__learning_rate': 0.5,
 'clf__estimator__n_estimators': 100,
 'tfidf__sublinear_tf': False}

In [33]:
cv.predict(['we are more than 50 people on the street.please help us find food and tent'])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.